# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.34.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'coupon-reco-hyper'
project_folder = './coupon-reco-proj-hyper'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
coupon-reco-hyper,quick-starts-ws-163001,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cc-sta-ds12-hp"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           min_nodes=1,
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# NOTE: update the key to match the dataset name
found = False
key = "in-vehicle-coupon-reco"
description_text = "In-vehicle coupon recommendation Data Set"

if key in ws.datasets.keys(): 
        found = True
        print("Dataset is already present in the workspace")
        dataset = ws.datasets[key] 

if not found:
        print("Loading the dataset from the location")
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

Loading the dataset from the location


,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,1,0,0,1,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Model, Environment

environment = Environment(name="azure-env")
conda_env = CondaDependencies()

# Needed packages and scripts
conda_env.add_conda_package("pandas")
conda_env.add_conda_package("numpy")
conda_env.add_conda_package("scikit-learn")
conda_env.add_pip_package("azureml-defaults")

# Adding dependencies to the created environment
environment.python.conda_dependencies=conda_env

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0,100,1000),
        '--max_iter': choice(10,25)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = "./",entry_script="train.py",compute_target=compute_target)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=5,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=early_termination_policy,
    estimator=estimator
)



'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [8]:
#TODO: Submit your experiment

from azureml.core.experiment import Experiment


# experiment=Experiment(ws,experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6e62d5f3-5855-4e7c-87cf-a2a73fa3cb92
Web View: https://ml.azure.com/runs/HD_6e62d5f3-5855-4e7c-87cf-a2a73fa3cb92?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-163001/workspaces/quick-starts-ws-163001&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-11-08T08:56:42.505004][API][INFO]Experiment created<END>\n""<START>[2021-11-08T08:56:43.148002][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-11-08T08:56:43.335946][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_6e62d5f3-5855-4e7c-87cf-a2a73fa3cb92
Web View: https://ml.azure.com/runs/HD_6e62d5f3-5855-4e7c-87cf-a2a73fa3cb92?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-163001/workspaces/quick-starts-ws-163001&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
import joblib
import os
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength: ',parameter_values[1])
print('\n Max iterations: ',parameter_values[3])

Best Run Id:  HD_6e62d5f3-5855-4e7c-87cf-a2a73fa3cb92_0

 Accuracy: 0.6666666666666666

 Regularization Strength:  50

 Max iterations:  10


In [11]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
coupon-reco-hyper,HD_6e62d5f3-5855-4e7c-87cf-a2a73fa3cb92_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
joblib.dump(parameter_values, filename='best-hyp-model.pkl')

['best-hyp-model.pkl']

In [13]:
from azureml.core import  Model

model_name = 'Coupon-Reco-HYP-model'

model = Model.register(workspace=ws,model_name = model_name, model_path = 'best-hyp-model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model Coupon-Reco-HYP-model
Coupon-Reco-HYP-model	Coupon-Reco-HYP-model:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

